# Reference atomic structures

This Notebook gets reference atomic crystal structures from various external sites and compares them to known crystal prototype structures.

**Library imports**

In [1]:
# Standard Python libraries
from __future__ import (print_function, division, absolute_import,
                        unicode_literals)
import os
import glob

# http://www.numpy.org/
import numpy as np

# https://pandas.pydata.org/
import pandas as pd

# https://github.com/usnistgov/atomman
import atomman as am
import atomman.lammps as lmp

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.8.2


## 1. Build unique sets of elements from included potentials

### Load database

In [2]:
database = iprPy.load_database('demo')

### Option #1: Get all elements lists from potentials

In [3]:
elements_set = set()
for potential_record in database.get_records(style='potential_LAMMPS'):
    potential = lmp.Potential(potential_record.content)
    elements = potential.elements()
    elements.sort()
    elements_set.add(' '.join(elements))

### Option #2: Only get elements lists for certain systems

In [3]:
elements_set = set()
elements_set.add('Cu Ni')

## 2. Fetch Materials Project reference structures

### Specify personal Materials Project API key

In [4]:
mp_api_key_location = 'C:\\users\\lmh1\\Documents\\Materials Project\\API key.txt'

with open(mp_api_key_location) as f:
    mp_api_key = f.read()

### Get structures from Materials Project

In [5]:
for elements in elements_set:
    iprPy.tools.get_mp_structures(elements.split(), api_key=mp_api_key)

## 3. Fetch OQMD reference structures

### Get structures from OQMD

In [6]:
for elements in elements_set:
    try:
        iprPy.tools.get_oqmd_structures(elements.split())
    except:
        print('Failed:', elements)

## 4. Run crystal_space_group calculations on the reference structures

In [3]:
calculation = iprPy.load_calculation('crystal_space_group')
run_directory = iprPy.load_run_directory('demo_1')

In [4]:
input_script = """

# Build load information based on prototype records
buildcombos                 crystalprototype load_file proto

# Build load information based on reference structures
buildcombos                 atomicreference load_file ref

# Specify reference buildcombos limiters (only build for element sets listed)
ref_elements                Mo

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
symmetryprecision           
primitivecell               
idealcell                   
"""
with open('input_script.in', 'w') as f:
    f.write(input_script)

In [5]:
calculation = iprPy.load_calculation('crystal_space_group')

with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)
    
database.prepare(run_directory, calculation, **input_dict)
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 518 of style calculation_crystal_space_group
 - 497 are complete
 - 21 still to run
 - 0 issued errors


In [6]:
database.runner(run_directory)
database.check_records(calculation.record_style)

Runner started with pid 10292
Didn't find an open simulation
No simulations left to run
In database style local at C:\Users\lmh1\Documents\calculations\ipr\demo :
- 518 of style calculation_crystal_space_group
 - 518 are complete
 - 0 still to run
 - 0 issued errors


## 5. Match prototypes to references

In [11]:
crystal_match_file = 'reference_prototype_match.csv'

### Retrieve finished calculation results

In [12]:
spg_records = database.get_records_df(style='calculation_crystal_space_group', full=True, flat=False, status='finished')

### Identify compositions

In [13]:
iprPy.analysis.assign_composition(spg_records, database)

### Split all spg records into references, prototypes and calculation relaxes

In [14]:
spg_records['record_type'] = 'calc'
spg_records.loc[(spg_records.load_file == spg_records.family + '.poscar'), 'record_type'] = 'reference'
spg_records.loc[(spg_records.load_file == spg_records.family + '.json'), 'record_type'] = 'prototype'

prototype_records = spg_records[spg_records.record_type == 'prototype']
reference_records = spg_records[spg_records.record_type == 'reference']
family_records = spg_records[(spg_records.record_type == 'prototype') | (spg_records.record_type == 'reference')]

calc_records = spg_records[spg_records.record_type == 'calc'].reset_index(drop=True)

### Create crystal_match_file

In [15]:
ref_proto_match = []

for ref_record in reference_records.itertuples():
    ref_proto_dict = {}
    ref_proto_dict['reference'] = ref_record.family
    ref_proto_dict['composition'] = ref_record.composition
    ref_proto_dict['site'], ref_proto_dict['number'] = ref_record.family.split('-')
    ref_proto_dict['number'] = int(ref_proto_dict['number'])
    
    # Find matching prototypes based on spg number and wykoffs
    prototype = np.nan
    
    # A1--Cu--fcc
    if (ref_record.spacegroup_number == 225 
    and ref_record.wykoff_fingerprint in ['a', 'b']):
        prototype = 'A1--Cu--fcc'
    
    # A2--W--bcc
    elif (ref_record.spacegroup_number == 229 
      and ref_record.wykoff_fingerprint in ['a']):
        prototype = 'A2--W--bcc'
    
    # A3'--alpha-La--double-hcp
    elif (ref_record.spacegroup_number == 194
      and ref_record.wykoff_fingerprint in ['ab', 'ac', 'ad']):
        prototype = "A3'--alpha-La--double-hcp"
    
    # A3--Mg--hcp
    elif (ref_record.spacegroup_number == 194
      and ref_record.wykoff_fingerprint in ['b', 'c', 'd']):
        prototype = 'A3--Mg--hcp'
        
    # A4--C--dc
    elif (ref_record.spacegroup_number == 227
      and ref_record.wykoff_fingerprint in ['a', 'b']):
        prototype = 'A4--C--dc'
    
    # A5--beta-Sn
    elif (ref_record.spacegroup_number == 141
      and ref_record.wykoff_fingerprint in ['a', 'b']):
        prototype = 'A5--beta-Sn'
    
    # A6--In--bct
    elif (ref_record.spacegroup_number == 139
      and ref_record.wykoff_fingerprint in ['a', 'b']):
        prototype = 'A6--In--bct'
        
    # A7--alpha-As
    elif (ref_record.spacegroup_number == 166
      and ref_record.wykoff_fingerprint in ['c']):
        prototype = 'A7--alpha-As'
        
    # A15--beta-W
    elif (ref_record.spacegroup_number == 223
      and ref_record.wykoff_fingerprint in ['ac', 'ad']):
        prototype = 'A15--beta-W'
    
    # A15--Cr3Si
    elif (ref_record.spacegroup_number == 223
      and ref_record.wykoff_fingerprint in ['a c', 'c a', 'a d', 'd a']):
        prototype = 'A15--Cr3Si'
    
    # Ah--alpha-Po--sc
    elif (ref_record.spacegroup_number == 221
      and ref_record.wykoff_fingerprint in ['a', 'b']):
        prototype = 'Ah--alpha-Po--sc'
    
    # B1--NaCl--rock-salt
    elif (ref_record.spacegroup_number == 225
      and ref_record.wykoff_fingerprint in ['a b', 'b a']):
        prototype = 'B1--NaCl--rock-salt'
    
    # B2--CsCl
    elif (ref_record.spacegroup_number == 221
      and ref_record.wykoff_fingerprint in ['a b', 'b a']):
        prototype = 'B2--CsCl'
    
    # B3--ZnS--cubic-zinc-blende
    elif (ref_record.spacegroup_number == 216
      and ref_record.wykoff_fingerprint in ['a c', 'c a', 'b d', 'd b', 'a d', 'd a', 'b c', 'c b']):
        prototype = 'B3--ZnS--cubic-zinc-blende'
    
    # C1--CaF2--fluorite
    elif (ref_record.spacegroup_number == 225
      and ref_record.wykoff_fingerprint in ['a c', 'c a', 'b c', 'c b']):
        prototype = 'C1--CaF2--fluorite'
    
    # D0_3--BiF3
    elif (ref_record.spacegroup_number == 225
      and ref_record.wykoff_fingerprint in ['a bc', 'bc a', 'b ac', 'ac b']):
        prototype = 'D0_3--BiF3'
    
    # L1_0--AuCu
    elif (ref_record.spacegroup_number == 123
      and ref_record.wykoff_fingerprint in ['a d', 'd a', 'b c', 'c b']):
        prototype = 'L1_0--AuCu'
    
    # L1_2--AuCu3
    elif (ref_record.spacegroup_number == 221
      and ref_record.wykoff_fingerprint in ['a c', 'c a', 'b d', 'd b']):
        prototype = 'L1_2--AuCu3'
    
    # L2_1--AlCu2Mn--heusler
    elif (ref_record.spacegroup_number == 225
      and ref_record.wykoff_fingerprint in ['a b c', 'b c a', 'c a b', 'a c b', 'c b a', 'b a c']):
        prototype = 'L2_1--AlCu2Mn--heusler'
    
    ref_proto_dict['prototype'] = prototype
    ref_proto_match.append(ref_proto_dict)
ref_proto_match = pd.DataFrame(ref_proto_match)
ref_proto_match = ref_proto_match.sort_values(['site', 'number']).reset_index()
ref_proto_match = ref_proto_match[['reference', 'prototype', 'composition']]
ref_proto_match.to_csv(crystal_match_file, index=False)

In [17]:
for protoname in sorted(prototype_records.family):
    print(protoname, len(ref_proto_match[ref_proto_match.prototype==protoname]))

A1--Cu--fcc 6
A15--Cr3Si 0
A15--beta-W 3
A2--W--bcc 5
A3'--alpha-La--double-hcp 5
A3--Mg--hcp 6
A4--C--dc 3
A5--beta-Sn 3
A6--In--bct 4
A7--alpha-As 0
Ah--alpha-Po--sc 0
B1--NaCl--rock-salt 0
B2--CsCl 0
B3--ZnS--cubic-zinc-blende 0
C1--CaF2--fluorite 0
D0_3--BiF3 0
L1_0--AuCu 0
L1_2--AuCu3 0
L2_1--AlCu2Mn--heusler 0
